In this notebook, we:

 * set up parameters
 * load images and display one
 * low-level filtering of the good versus bad images
 * todo: low-level re-alignment (seems not necessary)

In [1]:
%load_ext autoreload
%autoreload 2
from IPython.display import Image, display, Video

https://laurentperrinet.github.io/sciblog/posts/2020-08-09-nesting-jupyter-runs.html

In [2]:
#verb =  (__name__ == "__main__")
def has_parent():
    """
    https://stackoverflow.com/questions/48067529/ipython-run-magic-n-switch-not-working
    
    Return True if this notebook is being run by calling
    %run in another notebook, False otherwise.
    """
    try:
        __file__
        # __file__ has been defined, so this notebook is 
        # being run in a parent notebook
        return True

    except NameError:
        # __file__ has not been defined, so this notebook is 
        # not being run in a parent notebook
        return False
def do_verb():
    return not has_parent()

verb = do_verb()

In [3]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
phi = (np.sqrt(5)+1)/2

In [4]:
def init(args=[], date=None, DEBUG=1):
    if date is None:
        import datetime
        # https://en.wikipedia.org/wiki/ISO_8601
        date = datetime.datetime.now().date().isoformat()
    tag = f'{date}_TimeTeleScope'

    import argparse
    # Training settings
    parser = argparse.ArgumentParser(description='HollowGram')
    parser.add_argument('--tag', type=str, default=tag, help='unique ID')
    parser.add_argument('--date', type=str, default=date, help='date of the run')
    parser.add_argument('--in-path', type=str, default='../orchid', help='Folder to get images')
    parser.add_argument('--cache-path', type=str, default='cache_path', help='Folder to store intermediate images')
    parser.add_argument('--videos-path', type=str, default='videos', help='Folder to store videos')
    parser.add_argument('--figwidth', type=float, default=15, help='Width of figures')
    parser.add_argument('--ds', type=int, default=1, help='downsampling factor')
    parser.add_argument('--fps', type=float, default=12, help='FPS of output gif')
    parser.add_argument('--verb', type=bool, default=0, help='blah or not blah')
    args = parser.parse_args(args=[])

    return args

In [5]:
args = init()
args

Namespace(tag='2021-02-17_TimeTeleScope', date='2021-02-17', in_path='../orchid', cache_path='cache_path', videos_path='videos', figwidth=15, ds=1, fps=12, verb=0)

In [6]:
import os
class TimeTeleScopeLoad():
    def __init__(self, args):
        self.args = args
        if self.args.verb : print(f'Tagging our simulations with tag={self.args.tag}')
            
        os.makedirs(self.args.cache_path, exist_ok=True)
        os.makedirs(self.args.videos_path, exist_ok=True)

        self.figsize = (self.args.figwidth, self.args.figwidth/phi)

    # IMAGE
    def glob(self, gex='*.jpg'):
        """
        Globs images in a folder.

        Images should have a jpg extension and
        be in JPG format.
        
        Returns a list of image filenames.
        """
        fnames = []

        import glob
        # https://stackoverflow.com/questions/6773584/how-is-pythons-glob-glob-ordered
        for fname in sorted(glob.glob(f'{self.args.in_path}/{gex}')):
            fnames.append(fname)

        return fnames
    
    def glob_and_load(self, gex='2021-02-14*.jpg'):
        """
        Globs images in a folder.

        Images should have a jpg extension and
        be in JPG format.
        
        Returns a list of images.
        """
        imgs = {}

        import glob
        # https://stackoverflow.com/questions/6773584/how-is-pythons-glob-glob-ordered
        for fname in sorted(glob.glob(f'{self.args.in_path}/{gex}')):
            imgs[fname] = self.process(fname, do_preprend=False)

        return imgs


    def process(self, fname):
        # Preprocessing
        nx, ny, img = self.load(fname)
        if self.args.verb >= 20 : print('img min max', img.min(), img.max())
        return img/255.
    
    def load(self, fname):
        """
        Load an image.

        Range clipped to [0 ... 1]
        """
        import cv2
        if self.args.verb : print(f'Loading {fname}')
        im = cv2.imread(fname, cv2.IMREAD_UNCHANGED)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

        # resample by resizing image
        nx, ny, three = im.shape
        if self.args.ds > 1:
            # CAREFUL ! for a strange reason openCV inverts H and W ??
            im = cv2.resize(im, (ny//self.args.ds, nx//self.args.ds),
                            interpolation = cv2.INTER_AREA)
            nx, ny, three = im.shape
        assert(three==3)
        return nx, ny, im


In [7]:
ttl = TimeTeleScopeLoad(args)

In [8]:
fnames = ttl.glob()

In [9]:
print(f'{len(fnames)}=')

684=


In [10]:

def get_mask(sourceImage, sigma=.61803):
    
    N_X, N_Y, three = sourceImage.shape
    x, y, c = np.mgrid[-1:1:1j*N_X, -1:1:1j*N_Y, -1:1:1j*three]
    mask = np.exp(-.5* x**2 + y**2 / sigma**2)
    return mask
